In [ ]:
'''import stan
import arviz as az
import nest_asyncio
from itertools import chain
nest_asyncio.apply()

N = len(users)
K = len(groups)
D = len(groups)
Y = np.zeros(N)
X = np.zeros((N, D))

for user in users:
    # Filter data for user
    user_data = [row for row in data_for_regression if row[0] == user.id]
    
    final_entry = user_data[-1]
  
    if len(final_entry) - 2 < D:
        final_entry += [0] * (D - len(final_entry) + 2)

    Y[user.id - 1] = final_entry[1]
    X[user.id - 1] = final_entry[2:]

Y = Y.astype(int)

for row in X:
    row /= np.sum(row)

X = np.where(X == 0, 0.0001, X)

stan_code = """
data {
  int K;
  int N;
  int D;
  array[N] int Y;
  matrix[N, D] X;
}
parameters {
  matrix[D, K] beta;
}
model {
  matrix[N, K] x_beta = X * beta;

  to_vector(beta) ~ normal(0, 1);

  for (n in 1:N) {
    Y[n] ~ categorical_logit(x_beta[n]');

  }
}
"""
model = stan.build(stan_code, data={'K' : K, 'N' : N, 'D' : D, 'Y' : Y, 'X' : X.astype(float)})
fit = model.sample(num_chains=4, num_samples=1000)

az_data = az.from_pystan(posterior=fit)

print(az.summary(az_data, round_to=2, kind='stats')
      )
print(az.summary(az_data, round_to=2, kind='diagnostics'))

coefs = az_data.posterior.beta.mean(axis=(0,1))
conv = X @ np.array(coefs)
# set negative values to 0
conv = np.where(conv < 0, 0, conv)
# normalize
for row in conv:
    row /= np.sum(row)


# user preferences ?!?!
for i in range(len(conv)):
  plt.plot(conv[i])
'''